In [3]:
import pandas as pd
import random

# Define possible values
categories = ["Bills", "Groceries", "Delivery", "Others"]
urgencies = ["Low", "Medium", "High"]

# Generate 100 mock tasks
data = []
for i in range(100):
    category = random.choice(categories)
    urgency = random.choice(urgencies)
    deadline_hours = random.randint(1, 120)  # between 1h and 5 days
    
    # Simple rule for ground truth priority
    if urgency == "High" or deadline_hours < 24:
        priority = "High"
    elif urgency == "Medium" or deadline_hours < 72:
        priority = "Medium"
    else:
        priority = "Low"
    
    data.append({
        "task_id": i+1,
        "title": f"Task {i+1}",
        "category": category,
        "urgency": urgency,
        "deadline_hours": deadline_hours,
        "priority": priority
    })

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV (for reuse later)
df.to_csv("mock_tasks.csv", index=False)

df.head(10)  # preview first 10 tasks


,task_id,title,category,urgency,deadline_hours,priority
0,1,Task 1,Others,Medium,10,High
1,2,Task 2,Others,Low,48,Medium
2,3,Task 3,Delivery,Low,16,High
3,4,Task 4,Groceries,Medium,72,Medium
4,5,Task 5,Others,High,88,High
5,6,Task 6,Others,High,91,High
6,7,Task 7,Groceries,Low,13,High
7,8,Task 8,Groceries,Low,91,Low
8,9,Task 9,Delivery,High,23,High
9,10,Task 10,Bills,Low,19,High


In [4]:
import pandas as pd

df = pd.read_csv("mock_tasks.csv")
print(df.shape)
df.head()


(100, 6)


,task_id,title,category,urgency,deadline_hours,priority
0,1,Task 1,Others,Medium,10,High
1,2,Task 2,Others,Low,48,Medium
2,3,Task 3,Delivery,Low,16,High
3,4,Task 4,Groceries,Medium,72,Medium
4,5,Task 5,Others,High,88,High


In [5]:
from sklearn.preprocessing import LabelEncoder

# Copy dataframe to avoid messing up original
data = df.copy()

# Encode category and urgency
le_category = LabelEncoder()
le_urgency = LabelEncoder()
le_priority = LabelEncoder()

data["category_enc"] = le_category.fit_transform(data["category"])
data["urgency_enc"] = le_urgency.fit_transform(data["urgency"])
data["priority_enc"] = le_priority.fit_transform(data["priority"])  # target

data.head()


,task_id,title,category,urgency,deadline_hours,priority,category_enc,urgency_enc,priority_enc
0,1,Task 1,Others,Medium,10,High,3,2,0
1,2,Task 2,Others,Low,48,Medium,3,1,2
2,3,Task 3,Delivery,Low,16,High,1,1,0
3,4,Task 4,Groceries,Medium,72,Medium,2,2,2
4,5,Task 5,Others,High,88,High,3,0,0


In [6]:
X = data[["category_enc", "urgency_enc", "deadline_hours"]]
y = data["priority_enc"]


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [8]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [9]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le_priority.classes_))


Accuracy: 0.95

Classification Report:
               precision    recall  f1-score   support

        High       1.00      1.00      1.00         7
         Low       1.00      0.67      0.80         3
      Medium       0.91      1.00      0.95        10

    accuracy                           0.95        20
   macro avg       0.97      0.89      0.92        20
weighted avg       0.95      0.95      0.95        20



In [10]:
# Example: Grocery task, High urgency, deadline in 5 hours
sample_task = [[le_category.transform(["Groceries"])[0],
                le_urgency.transform(["High"])[0],
                5]]

pred = model.predict(sample_task)
print("Predicted priority:", le_priority.inverse_transform(pred)[0])


Predicted priority: High


C:\Users\User\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [12]:
import joblib

joblib.dump(model, "random_forest_model.joblib")
print("Model saved as random_forest_model.joblib")

Model saved as random_forest_model.joblib
